## CHUYÊN ĐỀ THỊ GIÁC MÁY TÍNH
### Giảng viên: PGS.TS. Lê Đình Duy
#### Học viên thực hiện: Trần Quang Hân         &emsp;     MSHV: CH1801027
<center> <h1>FACE RECOGNITION - MINI CHALLENGE<center>

### I - GIỚI THIỆU
##### 1. - Bài toán
<p>
 Bài toán nhận diện gương mặt ở lớp học, với mục tiêu xa là ứng dụng vào điểm danh cho lớp học chỉ với ảnh được chụp từ smartphone
    
-		Input: Ảnh chụp bằng smartphone.

-		Output: Ảnh với bounding box gương mặt và gán label cho gương mặt đó

-		Dataset: ảnh các gương mặt được cắt ra từ ảnh chụp smartphone, được bổ sung từ nhiều học viên trong lớp. (ví dụ học viên Trần Quang Hân, trong folder DATASET tạo folder HANTQ, bên trong để ảnh khuôn mặt của học viên đấy)


-		Bài làm thực hiện trên Jupyter NoteBook, python version 3.6. Sử dụng thuật toán K Neighbors Classifier.
</p>

##### 2. - Mục Lục

###### Phần I. Giới thiệu
Giới thiệu bài toán và mục lục.

###### Phần II. Cài đặt
Chi tiết các bước cài đặt bài toán, đánh giá kết quả cho bài làm.

###### Phần III. Tài liệu tham khảo


### II - CÀI ĐẶT

#### 1. - Đầu tiên ta khai báo các thư viện cần thiết

In [2]:
import face_recognition 
# thư viện face_recognition cung cấp các lệnh để nhận diện khuôn mặt
import math
# math là một thư viện toán, ta có thể sử dụng các toán có sẵn như  sqrt(),ceil(),...
from sklearn import neighbors
# sklearn là một thư viện mã nguồi mở dành cho máy học, chưa hầu hết các thuật toán máy học hiện đại nhất
# trong trường hợp này ta sử dụng thuật toán K-nearest neighbors
import os
# thư viện os cung cấp hàng loạt các hàm dùng cho việc giao tiếp với hệ điều hành
import os.path
# path thực hiện một số chức năng hữu ích trên tên đường dẫn. Để đọc hoặc ghi tệp, hãy xem open () và để truy cập hệ thống tệp.
import pickle
# pickle chuyển đổi một đối tượng Python sang dạng binary và ngược lại
from PIL import Image, ImageDraw,ImageFont
# Thư viện PIL hỗ trợ nhiều định dạng tập tin, và cung cấp khả năng xử lý hình ảnh và đồ hoạ.
from face_recognition.face_recognition_cli import image_files_in_folder
#hàm con image_files_in_folder trả về path các file trong folder, trong trường hợp này là path của image

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
# các đuôi ảnh mà ta xử lý trong project này

#### 2. - Hàm con huấn luyện KNN (k-nearest neighbors classifier for face recognition)

In [10]:

def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    
    """    
     (View in source code to see train_dir example tree structure)
    
    # Cấu trúc lưu trữ folder, file
     Structure:
        <train_dir>/
        ├── <person1>/
        │   ├── <somename1>.jpeg
        │   ├── <somename2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...
    """
    X = [] # khai X,y để lưu các giá trị trả về tạm thời
    y = []

    # Vòng qua từng người trong tập huấn luyện
    for class_dir in os.listdir(train_dir): #class_dir bây giờ được gán bằng tên folder (tên một người)
        #os.listdir để liệt kê các file hay folder có trong train_dir
        
        if not os.path.isdir(os.path.join(train_dir, class_dir)): #kiểm tra class_dir có phải là 1 folder không
            continue # nếu không phải là folder thì next class_dir khác

        # Lặp lại qua từng hình ảnh của folder ảnh người hiện tại
        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            
            image = face_recognition.load_image_file(img_path)#load file ảnh từ img_path gán vào image
            face_bounding_boxes = face_recognition.face_locations(image)
            #face_locations trả về 1 list các bộ vị trí mặt được tìm thấy theo thứ tự css (trên cùng, bên phải, dưới cùng, bên trái)

            if len(face_bounding_boxes) != 1:
                # nếu không có người hoặc nhiều hơn 1 người trong ảnh, qua ảnh tiếp theo và
                # in ra ảnh nào không phù hợp để loại bỏ.
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                # Thêm mã hóa khuôn mặt cho hình ảnh hiện tại vào tập huấn luyện                
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                # X lưu giá trị image, và location được mã hóa vào
                y.append(class_dir) # y lưu tên class(tên người đang xét)

    #Xác định có bao nhiêu hàng xóm để dùng tính trọng số trong phân loại KNN    
    if n_neighbors is None:
        # Nếu n_neighbors là None thì n_neighbors sẽ như công thức bên dưới, len(X) là độ dài của X, round là làm tròn, int lấy giá trị nguyên
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # Tạo và huấn luyện phân lớp KNN
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Lưu phân lớp KNN được huấn luyện
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

#### 3. - Hàm dự đoán sử dụng phân lớp KNN được huấn luyện

In [4]:
def predict(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.63):
    """           
    Nhận dạng khuôn mặt trong hình ảnh bằng cách sử dụng phân lớp KNN được đào tạo
    : param X_img_path: đường dẫn đến hình ảnh được nhận dạng
    : param knn_clf: (tùy chọn) một đối tượng phân lớplớp knn. nếu không được chỉ định, model_save_path phải được chỉ định.
    : param model_path: (tùy chọn) đường dẫn đến trình phân lớp knn ngâm. nếu không được chỉ định, model_save_path phải là knn_clf.
    : param distance_thr Ngưỡng: (tùy chọn) ngưỡng khoảng cách để phân loại khuôn mặt. 
          nó càng lớn, càng có nhiều cơ hội không phân lớp sai một người chưa biết là một người đã biết.
    : return: danh sách tên và vị trí khuôn mặt cho các khuôn mặt được nhận dạng trong hình ảnh: [(tên, hộp giới hạn), ...].
        Đối với khuôn mặt của những người không được nhận dạng, tên 'unknown' sẽ được trả lại.
    """
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        #kiểm tra định dạng file có nằm trong ALLOWED_EXTENSIONS hay không
        raise Exception("Invalid image path: {}".format(X_img_path))
        

    if knn_clf is None and model_path is None:
        #kiểm tra knn_clf và model_path có cùng None hay không. Nếu phải thì sẽ thông báo cung cấp.
        raise Exception("Must supply knn classifier either knn_clf or model_path")

    # Tải mô hình KNN được đào tạo (nếu được truyền vào)
    if knn_clf is None:
        with open(model_path, 'rb') as f: #mở và đọc file
            knn_clf = pickle.load(f)# mã hóa f và gán vào knn_clf

    # Tải tập tin hình ảnh và tìm vị trí các khuôn mặt
    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)
    
    # Nếu không tìm thấy khuôn mặt nào trong ảnh, trả về kết quả rỗng.
    if len(X_face_locations) == 0:
        return []
    
    # Tìm mã hóa cho các khuôn mặt trong hình ảnh thử nghiệm
    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)
 
    # Sử dụng mô hình KNN để tìm kết quả phù hợp nhất cho mặt kiểm tra
    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    #List Comprehensions
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    # Trả về dự đoán các lớp và loại bỏ các phân loại mà không nằm trong ngưỡng. 
    # Trong trường hợp này ngưỡng khooảng cách là 0.63, nếu nhỏ hơn sẽ được loại bỏ.
    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]

#### 4. - Hiển thị các nhãn trên hình ảnh

In [5]:
def show_prediction_labels_on_image(img_path, predictions):
    """    
    Hiển thị kết quả nhận dạng khuôn mặt một cách trực quan.
     : param img_path: đường dẫn đến hình ảnh được nhận dạng
     : param dự đoán: kết quả của chức năng dự đoán
     :Return:
    """
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:        
        # Vẽ hình chữ nhật quanh khuôn mặt, dùng module Pillow
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))
        
        # Dùng phông mặc định bitmap
        name = name.encode("UTF-8")
                
        # Vẽ nhãn với tên bên dưưới khuôn mặt
        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    # Xóa thư viện draw khỏi bộ nhớ
    del draw

    # Hiển thị kết quả ảnh
    pil_image.show()

#### 5. - Sau khi xong các hàm train, predict và show, ta vào hàm main để bắt đầu gọi các hàm đã viết để chạy project.

In [11]:
if __name__ == "__main__":
    # Bước 1: Huấn luyện phân lớp KNN và lưu vào trained_knn_model.clf
    # Mô hình sau khi huấn luyện và sẽ được lưu, nếu không có thay đổi trong dataset thì ta có thể bỏ qua bước này ở lần sau    
    print("Training KNN classifier...")
    classifier = train("DATASET", model_save_path="trained_knn_model.clf", n_neighbors=1)
    #n_neighbors=1 là xét 1 láng giềng gần nhất.
    print("Training complete!")        

    # Bước 2: Sử dụng phân lớp đã được huấn luyện để dự đoán các ảnh chưa biết khác.
    for image_file in os.listdir("test"):#os.listdir liệt kê các path trong folder test
        full_file_path = os.path.join("test", image_file)# nối path folder test với path của image_file, gán cho full_file_path

        print("Looking for faces in {}".format(image_file))#in định dạng của ảnh đang xét (image_file)

        # Ta dùng mô hình phân lớp đã huấn luyện để dự đoán những người có trong ảnh đang xét        
        predictions = predict(full_file_path, model_path="trained_knn_model.clf")

        # in kết quả ra Console
        for name, (top, right, bottom, left) in predictions:
            print("- Found {} at ({}, {})".format(name, left, top))

        # Hiển thị kết quả được phủ lên trên ảnh đang xét
        show_prediction_labels_on_image(os.path.join("test", image_file), predictions)

Training KNN classifier...
Training complete!
Looking for faces in IMG_5326.jpg
- Found THAODNT at (513, 1282)
- Found NHANTH at (1356, 1331)
- Found HANTQ at (2756, 1390)
- Found MINHHA at (3636, 841)
- Found VULQ at (2626, 1182)
- Found QUANVM at (3056, 924)
- Found VULQ at (3209, 868)
- Found MINHHA at (3417, 1148)
- Found DUYETLV at (2475, 927)
Looking for faces in IMG_5327.jpg
- Found MINHHA at (2862, 1243)
- Found TAIHPT at (765, 1538)
- Found VULQ at (163, 1035)
- Found VULQ at (772, 942)
- Found MINHHA at (1131, 873)
- Found DUYNN at (2122, 964)
- Found MYNH at (2615, 1336)
- Found DUYNN at (1560, 1113)
- Found QUANVM at (2103, 1538)
- Found MINHHA at (1788, 882)
- Found TANTD at (1801, 1099)
- Found TRUONGTX at (2088, 1228)
- Found MINHHA at (1061, 992)
- Found unknown at (3523, 1398)
- Found QUANVM at (2373, 1047)
- Found MINHHA at (1439, 932)
- Found HANTQ at (1250, 1023)
- Found QUANVM at (1596, 976)
Looking for faces in IMG_5328.jpg
- Found MINHHA at (3593, 1953)
- Found T

###### Hình ảnh minh họa
<img src="result/IMG_5326.jpg" />

#### 6. - Đánh giá kết quả.
##### > Thử nghiệm qua 7 tấm ảnh chụp trong buổi học.
<p>    
        	- Tỉ lệ tìm được khuôn mặt có trong ảnh là 83.4%.	
</p>
<p>        
       		- Tỉ lệ nhận dạng chính xác với các khuông mặt nhận dạng được là 49.1%.
</p>

#### III - TÀI LIỆU THAM KHẢO

[1] https://github.com/ageitgey/face_recognition/blob/master/examples/face_recognition_knn.py